# `geoplotlib`

`geoplotlib` is a small but very nice looking repository which provides a bunch of fast abstractions for geographic mapping.

In [1]:
import pandas as pd
import numpy as np
import geoplotlib
%matplotlib inline

In [2]:
import pyproj

In [3]:
sales = pd.read_csv("nyc_building_sales.csv")

C:\Anaconda3\envs\dummy_env_3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (23,26,28,29,70,71,86) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
pd.set_option("max_columns", 500)

In [5]:
sales.head(1)

,Unnamed: 0,Index,Borough,Block,Lot,SalePrice,LandSquareFeet,MarketValueSqFt,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,PolicePrct,HealthArea,SanitBoro,SanitDistrict,SanitSub,Address,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,Overlay1,Overlay2,SPDist1,SPDist2,LtdHeight,AllZoning1,AllZoning2,SplitZone,BldgClass,LandUse,Easements,OwnerType,OwnerName,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,StrgeArea,FactryArea,OtherArea,AreaSource,NumBldgs,NumFloors,UnitsRes,UnitsTotal,LotFront,LotDepth,BldgFront,BldgDepth,Ext,ProxCode,IrrLotCode,LotType,BsmtCode,AssessLand,AssessTot,ExemptLand,ExemptTot,YearBuilt,BuiltCode,YearAlter1,YearAlter2,HistDist,Landmark,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,BBL,CondoNo,Tract2010,XCoord,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,Version,CurFvT,NewFvT,CuravtA,AssessmentValueSqFt,EstPriorMarketValueSqFt,EstCurentMarketValueSqFt,ValueRatio
0,0,0,Bronx,2268.0,18.0,1800000.0,2500.0,221.538462,201.0,41.0,1002.0,7.0,8.0,10454.0,E083,40.0,4500.0,2.0,1.0,2A,532 EAST 142 STREET,R6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R6,NaN,N,C1,2.0,0.0,P,"HAXHARI, GAC",2500.0,8125.0,0.0,8125.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,4.0,7.0,7.0,25.0,100.0,0.0,0.0,NaN,2.0,N,5.0,2.0,2197.0,216878.0,0.0,0.0,2015.0,NaN,0.0,0.0,NaN,NaN,3.25,2.43,0.0,4.8,2.0,2.022680e+09,0.0,41.0,1007305.0,234328.0,6a,NaN,209S029,20901.0,NaN,0.0,NaN,1.0,16v1,143700.0,620769.0,2035.0,0.250462,17.686154,76.402338,2.899629


In [6]:
projstr = '+proj=lcc +lat_1=40.66666666666666 +lat_2=41.03333333333333 +lat_0=40.16666666666666 +lon_0=-74 +x_0=300000 +y_0=0 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs'
pnyc = pyproj.Proj(projstr,
            preserve_units=True)
coords = [pnyc(x, y, inverse=True) for x, y in zip(sales['XCoord'], sales['YCoord'])]

In [7]:
sales['XCoord'] = [coord[0] for coord in coords]
sales['YCoord'] = [coord[1] for coord in coords]

In [8]:
sales[['XCoord', 'YCoord']].sample(5)

,XCoord,YCoord
1678,-73.850081,40.856755
1523,-73.853486,40.845333
7937,-74.024192,40.630998
15374,-73.808698,40.764655
5515,-73.914251,40.701856


In [9]:
# sales['XCoord'] = [-coord for coord in sales['XCoord']]

## Dot map

Exactly what it says on the tin.

In [10]:
(sales['YCoord'] < 1).any()

False

Of note: `geoplotlib` expects the input to contain `lon` and `lat` columns, and will fail otherwise.

In [11]:
dat = sales[['XCoord', 'YCoord']].rename(columns = {'XCoord':'lon', 'YCoord': 'lat'}).dropna()
print(np.min(dat['lon']), np.max(dat['lon']))
print(np.min(dat['lat']), np.max(dat['lat']))
dat = dat[(dat['lon'] < 100) & (dat['lat'] < 100)]
print(np.min(dat['lon']), np.max(dat['lon']))
print(np.min(dat['lat']), np.max(dat['lat']))

-74.2538745258 1e+30
40.4988155161 1e+30
-74.2538745258 -73.7004708799
40.4988155161 40.9124086136


In [12]:
geoplotlib.dot(dat.sample(1000))
geoplotlib.show()

![alt text](Screen Shot 2016-06-27 at 11.42.02 PM.png "")

Simple enough, perhaps a bit unexciting. Can we do more?

## KDE map

Produces a density map. Note, however, that the maps is not cut to shorelines. Doing that probably requires some further work in QGIS (?).

In [13]:
# help(geoplotlib.utils.BoundingBox)

In [14]:
help(geoplotlib.kde)

Help on function kde in module geoplotlib:

kde(data, bw, cmap='hot', method='hist', scaling='sqrt', alpha=220, cut_below=None, clip_above=None, binsize=1, cmap_levels=10, show_colorbar=False)
    Kernel density estimation visualization
    
    :param data: data access object
    :param bw: kernel bandwidth (in screen coordinates)
    :param cmap: colormap
    :param method: if kde use KDEMultivariate from statsmodel, which provides a more accurate but much slower estimation.
        If hist, estimates density applying gaussian smoothing on a 2D histogram, which is much faster but less accurate
    :param scaling: colorscale, lin log or sqrt
    :param alpha: color alpha
    :param cut_below: densities below cut_below are not drawn
    :param clip_above: defines the max value for the colorscale
    :param binsize: size of the bins for hist estimator
    :param cmap_levels: discretize colors into cmap_levels levels
    :param show_colorbar: show colorbar



In [15]:
geoplotlib.kde(dat, bw=5, cut_below=1e-4)

# lowering clip_above changes the max value in the color scale
#geoplotlib.kde(data, bw=5, cut_below=1e-4, clip_above=.1)

# different bandwidths
#geoplotlib.kde(data, bw=20, cmap='PuBuGn', cut_below=1e-4)
#geoplotlib.kde(data, bw=2, cmap='PuBuGn', cut_below=1e-4)

# linear colorscale
#geoplotlib.kde(data, bw=5, cmap='jet', cut_below=1e-4, scaling='lin')

north, west, south, east = np.max(dat['lat']), np.min(dat['lon']), np.min(dat['lat']), np.max(dat['lon'])
geoplotlib.set_bbox(geoplotlib.utils.BoundingBox(north, west, south, east))
geoplotlib.show()

('smallest non-zero count', 7.1647865443840454e-10)
('max count:', 0.60091712044454904)


## Spatial map

Spatial maps are great. Functionally they make obvious sense. The API follows:

In [17]:
# import geoplotlib
# from geoplotlib.utils import read_csv


# data = read_csv('./data/flights.csv')
# geoplotlib.graph(data,
#                  src_lat='lat_departure',
#                  src_lon='lon_departure',
#                  dest_lat='lat_arrival',
#                  dest_lon='lon_arrival',
#                  color='hot_r',
#                  alpha=16,
#                  linewidth=2)
# geoplotlib.show()

![alt text](graph-flights.png "")